In [36]:
# Force reload modules (needed if you edited the code)
import importlib
import sys

# Remove cached modules
for mod in ['environment', 'models', 'game_state', 'config']:
    if mod in sys.modules:
        del sys.modules[mod]

print("Modules cleared - reimport in next cell")

Modules cleared - reimport in next cell


# Phase 3: Traitor / Deception / Poison / Evidence

Demonstrates:
- Random traitor assignment
- Traitor enhanced vision (sees all positions)
- Poison system (offer, symptoms, death)
- Sabotage mechanics
- Evidence logging
- Voting and elimination
- Win/loss conditions

In [37]:
import sys
sys.path.insert(0, '../marooned_env')

from environment import MaroonedEnv
from config import ActionType, ResourceType, ShipComponent, BASE_CAMP_POSITION
from models import Action, Position

In [38]:
# Initialize environment
env = MaroonedEnv(seed=42)
observations = env.reset()

print("ROLE ASSIGNMENT")
print(f"Traitor: {env.state.traitor_id}")
for name, sailor in env.state.sailors.items():
    print(f"{name}: {sailor.role.value}")

ROLE ASSIGNMENT
Traitor: Charlie
Alice: honest
Bob: honest
Charlie: traitor
Diana: honest
Eve: honest


In [39]:
# Test traitor enhanced vision
traitor_obs = env._generate_observation(env.state.traitor_id)
honest_obs = env._generate_observation('Alice' if env.state.traitor_id != 'Alice' else 'Bob')

print("TRAITOR ENHANCED VISION")
print(f"Traitor sees all positions: {traitor_obs.all_sailor_positions is not None}")
if traitor_obs.all_sailor_positions:
    for sailor, pos in traitor_obs.all_sailor_positions.items():
        print(f"  {sailor}: {pos.to_tuple()}")

print(f"\nHonest sailor sees all positions: {honest_obs.all_sailor_positions is not None}")

TRAITOR ENHANCED VISION
Traitor sees all positions: True
  Alice: (15, 15, <MapLevel.GROUND: 0>)
  Bob: (15, 15, <MapLevel.GROUND: 0>)
  Charlie: (15, 15, <MapLevel.GROUND: 0>)
  Diana: (15, 15, <MapLevel.GROUND: 0>)
  Eve: (15, 15, <MapLevel.GROUND: 0>)

Honest sailor sees all positions: False


In [40]:
# Test poison offering and progression
from config import MapLevel, PoisonState

# Give traitor a poison tablet
traitor = env.state.sailors[env.state.traitor_id]
from models import InventoryItem
traitor.add_to_backpack(ResourceType.POISON_TABLET, 1)

# Move victim next to traitor
victim_id = 'Alice' if env.state.traitor_id != 'Alice' else 'Bob'
victim = env.state.sailors[victim_id]
victim.position = traitor.position

# Traitor offers poison
print("POISON SYSTEM")
action = Action(
    sailor_id=env.state.traitor_id,
    action_type=ActionType.OFFER_FOOD,
    target_sailor=victim_id,
    resource_type=ResourceType.POISON_TABLET
)
result = env._execute_action(env.state.traitor_id, action)
print(f"Poison offered: {result['success']}")
print(f"Victim poisoned on day: {victim.poisoned_on_day}")
print(f"Poisoned by: {victim.poisoned_by}")

# Advance days to show poison progression
print("\nPoison progression:")
for day in range(1, 4):
    env.state.current_day = env.state.current_day + 1
    env._update_poison_states()
    print(f"Day {env.state.current_day}: {victim_id} state = {victim.poison_state.value}, alive = {victim.alive}")

POISON SYSTEM
Poison offered: True
Victim poisoned on day: 1
Poisoned by: Charlie

Poison progression:
Day 2: Alice state = early, alive = True
Day 3: Alice state = severe, alive = True
Day 4: Alice state = severe, alive = False


In [41]:
# Reset and test sabotage
env = MaroonedEnv(seed=100)
env.reset()

print("SABOTAGE SYSTEM")
traitor_id = env.state.traitor_id
print(f"Traitor: {traitor_id}")

# Setup: build ship progress first
env.state.ship_progress.components[ShipComponent.HULL].progress_percentage = 60
env.state.ship_progress.recalculate_total()
print(f"Initial HULL progress: {env.state.ship_progress.components[ShipComponent.HULL].progress_percentage}%")

# Move traitor to ship site
ship_pos = Position(*BASE_CAMP_POSITION)
env.state.sailors[traitor_id].position = ship_pos

# Traitor sabotages
action = Action(
    sailor_id=traitor_id,
    action_type=ActionType.SABOTAGE_SHIP,
    ship_component=ShipComponent.HULL
)
result = env._execute_action(traitor_id, action)
print(f"\nSabotage success: {result['success']}")
print(f"Damage: -{result.get('damage', 0)}%")
print(f"New HULL progress: {result.get('new_progress', 0)}%")
print(f"Evidence logged: {len(env.state.evidence_log.all_evidence)} entries")

SABOTAGE SYSTEM
Traitor: Diana
Initial HULL progress: 60%

Sabotage success: True
Damage: -35%
New HULL progress: 25%
Evidence logged: 1 entries


In [42]:
# Test evidence log
print("EVIDENCE LOG")
for i, evidence in enumerate(env.state.evidence_log.all_evidence, 1):
    print(f"{i}. Day {evidence.day} - {evidence.evidence_type.value}")
    print(f"   {evidence.description}")
    print(f"   Strength: {evidence.strength}/100")

EVIDENCE LOG
1. Day 1 - ship_sabotage
   Ship component hull was damaged! Progress reduced by 35%
   Strength: 95/100


In [43]:
# Test voting and elimination
env = MaroonedEnv(seed=200)
env.reset()

print("VOTING & ELIMINATION")
traitor_id = env.state.traitor_id
print(f"Traitor: {traitor_id}")
print(f"Living sailors: {sorted(env.state.living_sailors)}")

# Set phase to discussion (voting allowed)
env.state.current_phase = 'discussion'

# Call vote
caller = 'Alice' if traitor_id != 'Alice' else 'Bob'
action = Action(sailor_id=caller, action_type=ActionType.CALL_VOTE)
result = env._execute_action(caller, action)
print(f"\nVote initiated by {caller}: {result['success']}")

# All sailors vote for traitor (make a list copy to avoid set modification during iteration)
sailors_to_vote = list(env.state.living_sailors)
for sailor_id in sailors_to_vote:
    action = Action(
        sailor_id=sailor_id,
        action_type=ActionType.VOTE,
        vote_target=traitor_id
    )
    result = env._execute_action(sailor_id, action)

print(f"\nElimination result:")
print(f"  Eliminated: {result.get('eliminated')}")
print(f"  Was traitor: {result.get('was_traitor')}")
print(f"  Game over: {result.get('game_over')}")
print(f"  Winner: {result.get('winner')}")
print(f"\nRemaining sailors: {sorted(env.state.living_sailors)}")

VOTING & ELIMINATION
Traitor: Eve
Living sailors: ['Alice', 'Bob', 'Charlie', 'Diana', 'Eve']

Vote initiated by Alice: True

Elimination result:
  Eliminated: Eve
  Was traitor: True
  Game over: True
  Winner: sailors

Remaining sailors: ['Alice', 'Bob', 'Charlie', 'Diana']


In [44]:
# Test win conditions
env = MaroonedEnv(seed=300)
env.reset()

print("WIN/LOSS CONDITIONS")

# Test 1: Ship completion wins
env.state.ship_progress.total_percentage = 100
result = env._check_win_conditions()
print(f"Ship 100%: Winner = {result['winner'] if result else None}, Reason = {result['reason'] if result else None}")

# Test 2: Day 100 timeout (traitor wins)
env.state.ship_progress.total_percentage = 50
env.state.current_day = 100
result = env._check_win_conditions()
print(f"Day 100: Winner = {result['winner'] if result else None}, Reason = {result['reason'] if result else None}")

# Test 3: Too few sailors (traitor wins)
env.state.current_day = 10
env.state.living_sailors = {'Alice', 'Bob'}  # Only 2 left
result = env._check_win_conditions()
print(f"2 sailors: Winner = {result['winner'] if result else None}, Reason = {result['reason'] if result else None}")

WIN/LOSS CONDITIONS
Ship 100%: Winner = sailors, Reason = Ship complete
Day 100: Winner = traitor, Reason = Time expired
2 sailors: Winner = traitor, Reason = Too few sailors to complete ship
